# ANÁLISIS DEL DATASET "DIAMONDS".

## 1-CARGA DE DATOS, LIBRERÍAS DE PYTHON Y FUNCIONES AUXILIARES.

### 1.1-CARGA DE LIBRERÍAS DE PYTHON.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import re # Para poder utilizar expresiones regulares

### 1.2-CARGA DE DATOS.

In [2]:
# Archivo fuente de datos
file = './Data/diamonds.csv'

# Carga del archivo fuente y generación del DataFrame
df = pd.read_csv(file)

### 1.3-FUNCIONES AUXILIARES.

In [3]:
# Función para reemplazar valores (nulos o 'NaN'): por la mediana, en el caso de valores numéricos; por la moda, en el caso de valores categóricos.
# Entrada: pd.DataFrame / valor a sustituir (de tipo string|category|boolean o tipo NaN)
# Salida: pd.DataFrame / Se imprime en pantalla el número de reemplazos realizados por la moda y por la mediana. / Devuelve False si la no ha habido reemplazos.
def replace_values(df, valor):
    contador_modas = 0
    contador_medianas = 0
    for columna in df.columns:
        if df[columna].dtype == r'^(object|string|category|bool)$': # Los valores son de tipo texto ('object', 'string' o 'category') o boleanos.
            df[columna] = df[columna].fillna(df[columna].mode()[0])
            contador_modas += 1
        elif df[columna].dtype == r'^(int|float|UInt|Int)\d+$': # Los valores son de tipo numérico. 
            df[columna] = df[columna].fillna(df[columna].median())
            contador_medianas += 1
    if contador_modas == 0 & contador_medianas == 0: # No se han encontrado valores NaN en el DataFrame.
        print('No se han encontrado valores NaN que sustituir.')
        return False
    elif contador_modas > 0: # Se han reemplazado valores NaN por modas.
        print(f'Se han reemplazado valores NaN en {contador_modas} columnas por la moda de la columna.')
    elif contador_medianas > 0: # Se han reemplazado valores NaN por medianas.
        print(f'Se han reemplazado valores NaN en {contador_medianas} columnas por la mediana de la columna.')
    else:
        None
    return df

# Función para buscar coincidencias de caracteres las columnas de un DataFrame.
# Entrada: pd.DataFrame, string
# Salida: Lista con los nombres de las columnas del DataFrame donde se encontraron coincidencias. Si la lista está vacía, se devuelve False
def coincidencias_df (df, caracter):
    lista_coincidencias = [] # Lista para almacenar los nombres de las columnas donde se encontraron coincidencias.
    for columna in df.columns:
       if df[columna].astype(str).str.contains(str(caracter)).any(): # Se convierten los valores de la columna a string y se busca el caracter en la columna.
           lista_coincidencias.append(columna)
    if lista_coincidencias == []: # No se ha añadido ningún nombre de columna a la lista.
        print('No se han encontrado coincidencias.')
        return False
    else:
        return lista_coincidencias
    
# Función para sustituir valores 0 por la mediana de los valores de la columna con valores numéricos de un Data frame.
# Entrada: pd.DataFrame, lista de nombres de columnas con valores numéricos que contengan, al menos, un valor 0.
# Salida: pd.DataFrame | Se imprime un mensaje si alguna columna no existe en el DataFrame.
def reemplazar_ceros_con_mediana(df, columnas):
    # Iterar sobre las columnas especificadas
    for columna in columnas:
        if columna in df.columns:  # Verificar que la columna existe en el DataFrame.
            # Calcular la mediana excluyendo los ceros a sustituir, para que no influyan en el cálculo de la mediana.
            mediana_sin_ceros = df.loc[df[columna] != 0, columna].median()
            # Reemplazar los ceros por la mediana.
            df[columna] = df[columna].replace(0, mediana_sin_ceros)
        else:
            print(f"La columna '{columna}' no existe en el DataFrame.")
    
    return df



## 2-ANÁLISIS PRELIMINAR DEL DATAFRAME.

In [ ]:

lista = coincidencias_df(df, np.nan) # Como el caracter '?' es un caracter especial, se debe anteponer el caracter '\' para que sea reconocido como un caracter normal.
lista


In [ ]:
replace_nan(df)

### 2.1-ANÁLISIS DE LA ESTRUCTURA DEL DATAFRAME.

#### 2.1.1-VISUALIZACIÓN PREVIA DEL DATAFRAME.

In [ ]:
df.head(5)

In [ ]:
df[25000::5]

In [ ]:
df.tail(5)

Como se puede observar en las tablas superiores, el DataFrame cargado consta de 53.940 filas y 10 columnas, de las cuales, 7 columnas son numéricas y 3 columnas son categóricas, en principio.

### 2.2-ANÁLISIS DE LA INCONSISTENCIA DE DATOS.

In [ ]:
df.info()

Al ampliar la información del DataFrame, como se puede apreciar en la tabla superior, se indica que el DataFrame tiene 53.940 filas. Sin embargo, existe una única columna (la 4 - 'depth') en cuyas observaciones no existe ningún valor nulo. En el resto de columnas, se han detectado valores nulos: existen columnas con 1 valor nulo ('carat', 'y', 'z'), 2 valores nulos ('color', 'clarity', 'table' y 'price') y 3 valores nulos ('cut').
<p>Por otra parte, algunas columnas no tienen el tipo de dato que se esperaba a priori. Por ejemplo, las columnas 'price' y 'x', en principio, se esperaban que contuvieran valores numéricos y se han detectado valores de tipo 'object', por lo que los valores de las columnas indicadas pueden haberse cargado como tipo 'string'.
<P>Por todo ello, se deduce que existen valores inconsistentes en las columnas mencionadas que hay que tratar adecuadamente. 
<p>En principio, las cantidades de estas inconsitencias encontradas no son muy numerosas. Dada la cantidad total de las filas del DataFrame, las inconsistencias no parecen tener un peso importante en el conjunto del mismo, y una de las opciones podría ser eliminar directamente las filas afectadas. No obstante, es preferible evitar su eliminación (afín de no perder información) y las filas/columnas afectadas se van a tratar adecuadamente. Para ello, a continuación se procede al análisis de cada columna afectada.

### 2.3-ANÁLISIS POR COLUMNAS.

#### 2.3.1-COLUMNA: 'carat'.

In [ ]:
# Se imprimen los valores únicos de la columna. Se aprecia que el último valor obtenido es de tipo 'NaN
print(df['carat'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['carat'].value_counts(dropna=False, sort=True, ascending=True)

A parte de los valores NaN, no se aprecia ningún otro valor inconsistente en la columna.

In [ ]:
# Se imprime la línea que contiene el valor 'NaN' en la columna 'carat'
print(df[df['carat'].isnull()])

In [ ]:
filas_valor_cero = df[df['carat']==0]
filas_valor_cero

#### 2.3.2-COLUMNA: 'cut'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['cut'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['cut'].value_counts(dropna=False)

Se observa que, además de los 3 valores NaN, también aparece un valor '?', que no se correspondería con un valor esperado en la columna 'cut'.

In [ ]:
# Se imprime la fila que contiene el caracter '?' en la columna 'cut'
df[df['cut'].str.contains(r'\?', na=False)]

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna 'cut'
print(df[df['cut'].isnull()])


In [ ]:
filas_valor_cero = df[df['cut'].str.startswith('0', na=False)]
filas_valor_cero

No hay valores '0'.

#### 2.3.3-COLUMNA: 'color'.

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
print(df['color'].value_counts(dropna=False))

Se observa que, además de los 2 valores 'NaN', también aparece un valor '?' no esperado en la columna.

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna 'color'
print(df[df['color'].isnull()])


In [ ]:
filas_valor_cero = df[df['color'].str.startswith('0', na=False)]
filas_valor_cero

#### 2.3.4-COLUMNA: 'clarity'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['clarity'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['clarity'].value_counts(dropna=False)

En este caso, aparecen 2 valores NaN.

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna 'clarity'
print(df[df['clarity'].isnull()])


In [ ]:
filas_valor_cero = df[df['clarity'].str.startswith('0', na=False)]
filas_valor_cero

#### 2.3.5-COLUMNA: 'table'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['table'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['table'].value_counts(dropna=False)

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['table'].isnull()])


Aquí aparecen otros 2 valores NaN.

In [ ]:
filas_valor_cero = df[df['table']==0]
filas_valor_cero

#### 2.3.6-COLUMNA: 'price'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['price'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['price'].value_counts(dropna=False)

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['price'].isnull()])


Esta columna 'price' contiene valores que representan precios (en formato string) de diamantes. Dado que un precio '0' puede representar un valor anómalo no esperado, se busca en la columna si existe algún valor '0' o '0.'. Dando como resultado que no existe ningún valor de este tipo. En un apartado posterior, se realizará la conversión de tipo de dato a un tipo numérico más adecuado.

In [ ]:
print(('0' in df['price']) | ('0.' in df['price']))


#### 2.3.7-COLUMNA: 'x'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['x'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['x'].value_counts(dropna=False)

Aparece un valor '?'.

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['x'].isnull()])


In [ ]:
filas_valor_cero = df[df['x'].str.startswith('0', na=False)]
filas_valor_cero

En esta columna, también aparecen varios valores '0', que deberán ser tratados adecuadamente.

#### 2.3.8-COLUMNA: 'y'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['y'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['y'].value_counts(dropna=False)

In [ ]:
filas_valor_cero = df[df['y']==0]
filas_valor_cero

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['y'].isnull()])


#### 2.3.9-COLUMNA: 'z'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['z'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['z'].value_counts(dropna=False)

In [ ]:
filas_valor_cero = df[df['z']==0]
filas_valor_cero

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['z'].isnull()])


#### 2.3.10-COLUMNA: 'depth'.

In [ ]:
# Se imprimen los valores únicos de la columna.
print(df['depth'].unique())

In [ ]:
# Se imprimen los valores únicos de la columna junto a su frecuencia de aparición.
df['depth'].value_counts(dropna=False)

In [ ]:
# Se imprimen las líneas que contienen el valor 'NaN' en la columna.
print(df[df['depth'].isnull()])


La columna 'depth' no presenta ni valores 'NaN' ni ningún otro tipo de dato inconsistente.

### 2.4-LIMPIEZA, TRATAMIENTO DE DATOS NULOS O INCONSISTENTES Y CAMBIO DE TIPOS DE DATO .

Resúmen de las inconsistencias detectadas.

<table style="width:20%">
<tr>
<th></th>
<th>CARAT</th>
<th>CUT</th>
<th>COLOR</th>
<th>CLARITY</th>
<th>DEPTH</th>
<th>TABLE</th>
<th>PRICE</th>
<th>X</th>
<th>Y</th>
<th>Z</th>
</tr>
<tr>
<td>TIPO DATO Actual</td>
<td>FLOAT64</td>
<td>OBJECT</td>
<td>OBJECT</td>
<td>OBJECT</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
<td>OBJECT</td>
<td>OBJECT</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
</tr>

<tr>
<td>TIPO DATO Óptimo</td>
<td>FLOAT64</td>
<td>CATEGORY</td>
<td>OBJECT</td>
<td>CATEGORY</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
<td>FLOAT64</td>
</tr>

<tr>
<td>NaN</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
</tr>

<tr>
<td>?</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>NO</td>
<td>NO</td>
</tr>

<tr>
<td>0</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>NO</td>
<td>SÍ</td>
<td>SÍ</td>
<td>SÍ</td>
</tr>
</table>


De los datos inconsistentes detectados, tal y como aparecen en la tabla superior, el proceso que se va a realizar es el siguiente:
<ol>
<li>Sustituir los valores '?' por otro valor, por ejemplo 'NaN', para que se permita realizar adecuadamente la conversión de tipos de datos.</li>
<li>Realizar la conversión de tipo de datos.</li>
<li>Sustituir los valores '0' y '0.' por las medianas</li>
<li>Sustituir los valores 'NaN' por las medianas (valores numéricos) o por las modas (valores categóricos).</li>
</ol>

Se va a sustituir el valor 'NaN' por la mediana de la columna.

In [ ]:
# Se obtienen las columnas con, al menos, un valor NaN
columnas_nan = df.columns[df.isnull().any()]
print("Columnas con valores NaN:", columnas_nan)

# Reemplazar NaN con la mediana para columnas numéricas y la moda para categóricas
for columna in columnas_nan:
    if df[columna].dtype in ['float64', 'int64']:  # Si la columna es numérica
        median = df[columna].median()
        df[columna].fillna(median, inplace=True) # Se reemplaza NaN por la mediana
    else:  # Si la columna es categórica o string ('object')
        mode = df[columna].mode().iloc[0]  # Se selecciona la primera moda
        df[columna].fillna(mode, inplace=True) # Se reemplaza NaN por la moda

# Resultado
""" print("\nDataFrame después de reemplazar valores NaN:")
print(df) """
df.info()


In [ ]:
# Se reemplazan los carácteres '?' con NaN en la columna
df['cut'] = df['cut'].replace('?', np.nan)
df.replace('?', np.nan)